In [34]:
import sys
sys.path.append("..")
from lib.util import add_race_eth,  add_income_group
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1550
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [35]:
add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]

In [36]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = df[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('BANK OF AMERICA')][['respondent_id','respondent_name']].drop_duplicates()

,respondent_id,respondent_name
4398,0000012773,BANK OF AMERICAN FORK
17927,0000014740,FIRST NATIONAL BANK OF AMERICA
263517,0000480228,"BANK OF AMERICA, N.A."


# BoA
- **Missing: Minority Status, Tract/BNA**

In [37]:
df = df[df.respondent_id=='0000480228']

###
# Action type == ALL
###
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

### By Loan Purpose and Type

df['loan_purtype'] = '_'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] == 1)] = 'Purchase - Conventional'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] > 1)] = 'Purchase - Goverment'
df['loan_purtype'][df['loan_purpose'] == 2] = 'Home Improvement'
df['loan_purtype'][df['loan_purpose'] == 3] = 'Refinancing'

loanpt = df.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df['applicant_income_percent_median'] = 100*df.applicant_income/df.applicant_income.median()
df['applicant_income_cat'] = 'Income Not Available'
df['applicant_income_cat'][df['applicant_income_percent_median'] < 50] = 'Low'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 50) & (df['applicant_income_percent_median'] < 80)] = 'Moderate'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 80) & (df['applicant_income_percent_median'] < 120)] = 'Middle'
df['applicant_income_cat'][df['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications (1)'])
totalapp

Total Applications (1)          \
                                                                count  %count   
Loan Purpose and Type Home Improvement                            103     1.0   
                      Purchase - Conventional                    1022    10.3   
                      Purchase - Goverment                        550     5.5   
                      Refinancing                                8271    83.2   
Applicant Race/Eth    aian                                         40     0.4   
                      as                                         1023    10.3   
                      b                                           201     2.0   
                      h                                          1463    14.7   
                      pi                                           81     0.8   
                      w                                          5668    57.0   
Applicant Income      Income Not Available                       1462    14.7   
                      Low                                        1495    15.0   
                      Middle                                     1699    17.1   
                      Moderate                                   1828    18.4   
                      Upper                                      3462    34.8   
Applicant Sex         1                                          6979    70.2   
                      2                                          2324    23.4   
                      3                                           620     6.2   
                      4                                            23     0.2   

                                                                 
                                                amount  %amount  
Loan Purpose and Type Home Improvement           32417      0.9  
                      Purchase - Conventional   484014     13.9  
                      Purchase - Goverment      174186      5.0  
                      Refinancing              2783718     80.1  
Applicant Race/Eth    aian                       11999      0.3  
                      as                        329398      9.5  
                      b                          56321      1.6  
                      h                         354973     10.2  
                      pi                         22402      0.6  
                      w                        2052423     59.1  
Applicant Income      Income Not Available      504627     14.5  
                      Low                       275911      7.9  
                      Middle                    484262     13.9  
                      Moderate                  419810     12.1  
                      Upper                    1789725     51.5  
Applicant Sex         1                        2524987     72.7  
                      2                         645036     18.6  
                      3                         255241      7.3  
                      4                          49071      1.4

In [38]:
#######
# Action Type =1
#######
df1 = df[df.action_type==1]
totalcount = df1.loan_amount.count()
totalamount = df1.loan_amount.sum()

### By Loan Purpose and Type
df1['loan_purtype'] = '_'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] == 1)] = 'Purchase - Conventional'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] > 1)] = 'Purchase - Goverment'
df1['loan_purtype'][df1['loan_purpose'] == 2] = 'Home Improvement'
df1['loan_purtype'][df1['loan_purpose'] == 3] = 'Refinancing'

loanpt = df1.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df1.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]


###By Applicant Income

df1['applicant_income_percent_median'] = 100*df1.applicant_income/df1.applicant_income.median()
df1['applicant_income_cat'] = 'Income Not Available'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] < 50] = 'Low'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 50) & (df1['applicant_income_percent_median'] < 80)] = 'Moderate'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 80) & (df1['applicant_income_percent_median'] < 120)] = 'Middle'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df1.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df1.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp1 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp1.ix[:, ["count","%count","amount","%amount",]]]
totalapp1 = pd.concat(pieces, axis=1, keys=['Loan originated'])
totalapp1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan originated          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                      52     0.8   
                      Purchase - Conventional              622     9.8   
                      Purchase - Goverment                 243     3.8   
                      Refinancing                         5439    85.6   
Applicant Race/Eth    aian                                  22     0.3   
                      as                                   654    10.3   
                      b                                    133     2.1   
                      h                                    829    13.0   
                      pi                                    49     0.8   
                      w                                   3721    58.5   
Applicant Income      Income Not Available                1005    15.8   
                      Low                                  965    15.2   
                      Middle                              1136    17.9   
                      Moderate                            1095    17.2   
                      Upper                               2155    33.9   
Applicant Sex         1                                   4546    71.5   
                      2                                   1414    22.2   
                      3                                    381     6.0   
                      4                                     15     0.2   

                                                                 
                                                amount  %amount  
Loan Purpose and Type Home Improvement           17532      0.8  
                      Purchase - Conventional   294449     13.0  
                      Purchase - Goverment       81031      3.6  
                      Refinancing              1867375     82.6  
Applicant Race/Eth    aian                        6991      0.3  
                      as                        213490      9.4  
                      b                          37938      1.7  
                      h                         205775      9.1  
                      pi                         14324      0.6  
                      w                        1358351     60.1  
Applicant Income      Income Not Available      335716     14.9  
                      Low                       184930      8.2  
                      Middle                    322746     14.3  
                      Moderate                  252560     11.2  
                      Upper                    1164435     51.5  
Applicant Sex         1                        1675143     74.1  
                      2                         404315     17.9  
                      3                         162362      7.2  
                      4                          18567      0.8

In [39]:
#######
# Action Type =2 
#######
df2 = df[df.action_type==2]
totalcount = df2.loan_amount.count()
totalamount = df2.loan_amount.sum()

### By Loan Purpose and Type

df2['loan_purtype'] = '_'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] == 1)] = 'Purchase - Conventional'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] > 1)] = 'Purchase - Goverment'
df2['loan_purtype'][df2['loan_purpose'] == 2] = 'Home Improvement'
df2['loan_purtype'][df2['loan_purpose'] == 3] = 'Refinancing'

loanpt = df2.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df2.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df2['applicant_income_percent_median'] = 100*df2.applicant_income/df2.applicant_income.median()
df2['applicant_income_cat'] = 'Income Not Available'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] < 50] = 'Low'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 50) & (df2['applicant_income_percent_median'] < 80)] = 'Moderate'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 80) & (df2['applicant_income_percent_median'] < 120)] = 'Middle'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df2.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df2.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp2 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp2.ix[:, ["count","%count","amount","%amount",]]]
totalapp2 = pd.concat(pieces, axis=1, keys=['App approved but not accepted'])
totalapp2

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App approved but not accepted  \
                                                                       count   
Loan Purpose and Type Home Improvement                                     1   
                      Purchase - Conventional                             43   
                      Refinancing                                        214   
Applicant Race/Eth    aian                                                 1   
                      as                                                  27   
                      b                                                    1   
                      h                                                   23   
                      pi                                                   3   
                      w                                                  154   
Applicant Income      Income Not Available                                33   
                      Low                                                 48   
                      Middle                                              34   
                      Moderate                                            47   
                      Upper                                               96   
Applicant Sex         1                                                  160   
                      2                                                   72   
                      3                                                   25   
                      4                                                    1   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            0.4     546      0.5  
                      Purchase - Conventional    16.7   31246     30.1  
                      Refinancing                82.9   71990     69.4  
Applicant Race/Eth    aian                        0.4     310      0.3  
                      as                         10.5    7890      7.6  
                      b                           0.4     175      0.2  
                      h                           8.9    5658      5.5  
                      pi                          1.2     966      0.9  
                      w                          59.7   64096     61.8  
Applicant Income      Income Not Available       12.8   12011     11.6  
                      Low                        18.6    9697      9.3  
                      Middle                     13.2   11598     11.2  
                      Moderate                   18.2   11009     10.6  
                      Upper                      37.2   59467     57.3  
Applicant Sex         1                          62.0   68462     66.0  
                      2                          27.9   20154     19.4  
                      3                           9.7   13866     13.4  
                      4                           0.4    1300      1.3

In [40]:
#######
# Action Type =3 
#######
df3 = df[df.action_type==3]
totalcount = df3.loan_amount.count()
totalamount = df3.loan_amount.sum()

### By Loan Purpose and Type

df3['loan_purtype'] = '_'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] == 1)] = 'Purchase - Conventional'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] > 1)] = 'Purchase - Goverment'
df3['loan_purtype'][df3['loan_purpose'] == 2] = 'Home Improvement'
df3['loan_purtype'][df3['loan_purpose'] == 3] = 'Refinancing'

loanpt = df3.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df3.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df3['applicant_income_percent_median'] = 100*df3.applicant_income/df3.applicant_income.median()
df3['applicant_income_cat'] = 'Income Not Available'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] < 50] = 'Low'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 50) & (df3['applicant_income_percent_median'] < 80)] = 'Moderate'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 80) & (df3['applicant_income_percent_median'] < 120)] = 'Middle'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df3.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df3.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp3 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp3.ix[:, ["count","%count","amount","%amount",]]]
totalapp3 = pd.concat(pieces, axis=1, keys=['App denied by financial institution'])
totalapp3

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App denied by financial institution  \
                                                                             count   
Loan Purpose and Type Home Improvement                                          35   
                      Purchase - Conventional                                  204   
                      Purchase - Goverment                                     100   
                      Refinancing                                             1577   
Applicant Race/Eth    aian                                                       8   
                      as                                                       203   
                      b                                                         36   
                      h                                                        331   
                      pi                                                        12   
                      w                                                       1032   
Applicant Income      Income Not Available                                     244   
                      Low                                                      325   
                      Middle                                                   324   
                      Moderate                                                 358   
                      Upper                                                    665   
Applicant Sex         1                                                       1256   
                      2                                                        514   
                      3                                                        143   
                      4                                                          3   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            1.8   10083      1.6  
                      Purchase - Conventional    10.6   80438     12.4  
                      Purchase - Goverment        5.2   32482      5.0  
                      Refinancing                82.3  524407     81.0  
Applicant Race/Eth    aian                        0.4    1679      0.3  
                      as                         10.6   58954      9.1  
                      b                           1.9    9744      1.5  
                      h                          17.3   79187     12.2  
                      pi                          0.6    2879      0.4  
                      w                          53.9  378001     58.4  
Applicant Income      Income Not Available       12.7   80547     12.4  
                      Low                        17.0   57611      8.9  
                      Middle                     16.9   90574     14.0  
                      Moderate                   18.7   84134     13.0  
                      Upper                      34.7  334544     51.7  
Applicant Sex         1                          65.6  443748     68.5  
                      2                          26.8  141566     21.9  
                      3                           7.5   52712      8.1  
                      4                           0.2    9384      1.4

In [41]:
#######
# Action Type =4
#######
df4 = df[df.action_type==4]
totalcount = df4.loan_amount.count()
totalamount = df4.loan_amount.sum()

### By Loan Purpose and Type

df4['loan_purtype'] = '_'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] == 1)] = 'Purchase - Conventional'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] > 1)] = 'Purchase - Goverment'
df4['loan_purtype'][df4['loan_purpose'] == 2] = 'Home Improvement'
df4['loan_purtype'][df4['loan_purpose'] == 3] = 'Refinancing'

loanpt = df4.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df4.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df4['applicant_income_percent_median'] = 100*df4.applicant_income/df4.applicant_income.median()
df4['applicant_income_cat'] = 'Income Not Available'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] < 50] = 'Low'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 50) & (df4['applicant_income_percent_median'] < 80)] = 'Moderate'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 80) & (df4['applicant_income_percent_median'] < 120)] = 'Middle'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df4.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df4.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp4 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp4.ix[:, ["count","%count","amount","%amount",]]]
totalapp4 = pd.concat(pieces, axis=1, keys=['App withdrawn by applicant'])
totalapp4

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App withdrawn by applicant  \
                                                                    count   
Loan Purpose and Type Home Improvement                                  6   
                      Purchase - Conventional                         115   
                      Purchase - Goverment                             48   
                      Refinancing                                     601   
Applicant Race/Eth    aian                                              5   
                      as                                               88   
                      b                                                12   
                      h                                               136   
                      pi                                                7   
                      w                                               422   
Applicant Income      Income Not Available                             90   
                      Low                                             114   
                      Middle                                          140   
                      Moderate                                        146   
                      Upper                                           280   
Applicant Sex         1                                               546   
                      2                                               180   
                      3                                                41   
                      4                                                 3   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            0.8     738      0.3  
                      Purchase - Conventional    14.9   61614     24.2  
                      Purchase - Goverment        6.2   16806      6.6  
                      Refinancing                78.1  175489     68.9  
Applicant Race/Eth    aian                        0.6    1292      0.5  
                      as                         11.4   28644     11.2  
                      b                           1.6    3165      1.2  
                      h                          17.7   29441     11.6  
                      pi                          0.9    1379      0.5  
                      w                          54.8  139075     54.6  
Applicant Income      Income Not Available       11.7   44060     17.3  
                      Low                        14.8   18562      7.3  
                      Middle                     18.2   38203     15.0  
                      Moderate                   19.0   32035     12.6  
                      Upper                      36.4  121787     47.8  
Applicant Sex         1                          70.9  179212     70.4  
                      2                          23.4   42233     16.6  
                      3                           5.3   14882      5.8  
                      4                           0.4   18320      7.2

In [42]:
#######
# Action Type =5
#######

df5 = df[df.action_type==5]
totalcount = df5.loan_amount.count()
totalamount = df5.loan_amount.sum()

### By Loan Purpose and Type

df5['loan_purtype'] = '_'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] == 1)] = 'Purchase - Conventional'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] > 1)] = 'Purchase - Goverment'
df5['loan_purtype'][df5['loan_purpose'] == 2] = 'Home Improvement'
df5['loan_purtype'][df5['loan_purpose'] == 3] = 'Refinancing'

loanpt = df5.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df5.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df5['applicant_income_percent_median'] = 100*df5.applicant_income/df5.applicant_income.median()
df5['applicant_income_cat'] = 'Income Not Available'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] < 50] = 'Low'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 50) & (df5['applicant_income_percent_median'] < 80)] = 'Moderate'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 80) & (df5['applicant_income_percent_median'] < 120)] = 'Middle'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df5.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df5.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp5 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp5.ix[:, ["count","%count","amount","%amount",]]]
totalapp5 = pd.concat(pieces, axis=1, keys=['File closed for incompleteness'])
totalapp5

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


File closed for incompleteness  \
                                                                        count   
Loan Purpose and Type Home Improvement                                      6   
                      Purchase - Conventional                              13   
                      Purchase - Goverment                                  5   
                      Refinancing                                         296   
Applicant Race/Eth    aian                                                  1   
                      as                                                   36   
                      b                                                    10   
                      h                                                    66   
                      pi                                                    4   
                      w                                                   164   
Applicant Income      Income Not Available                                 47   
                      Low                                                  48   
                      Middle                                               42   
                      Moderate                                             64   
                      Upper                                               119   
Applicant Sex         1                                                   228   
                      2                                                    73   
                      3                                                    18   
                      4                                                     1   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            1.9    2554      2.2  
                      Purchase - Conventional     4.1    4072      3.5  
                      Purchase - Goverment        1.6    1701      1.5  
                      Refinancing                92.5  106825     92.8  
Applicant Race/Eth    aian                        0.3     227      0.2  
                      as                         11.2   15054     13.1  
                      b                           3.1    2870      2.5  
                      h                          20.6   15568     13.5  
                      pi                          1.2    1192      1.0  
                      w                          51.2   62328     54.1  
Applicant Income      Income Not Available       14.7   16762     14.6  
                      Low                        15.0    8186      7.1  
                      Middle                     13.1   11738     10.2  
                      Moderate                   20.0   14219     12.3  
                      Upper                      37.2   64247     55.8  
Applicant Sex         1                          71.2   88855     77.2  
                      2                          22.8   17452     15.2  
                      3                           5.6    7345      6.4  
                      4                           0.3    1500      1.3

In [62]:
#######
# Action Type = 6
#######

df6 = df[df.action_type==6]
totalcount = df6.loan_amount.count()
totalamount = df6.loan_amount.sum()

### By Loan Purpose and Type

df6['loan_purtype'] = '_'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] == 1)] = 'Purchase - Conventional'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] > 1)] = 'Purchase - Goverment'
df6['loan_purtype'][df6['loan_purpose'] == 2] = 'Home Improvement'
df6['loan_purtype'][df6['loan_purpose'] == 3] = 'Refinancing'

loanpt = df6.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df6.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df6['applicant_income_percent_median'] = 100*df6.applicant_income/df6.applicant_income.median()
df6['applicant_income_cat'] = 'Income Not Available'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] < 50] = 'Low'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 50) & (df6['applicant_income_percent_median'] < 80)] = 'Moderate'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 80) & (df6['applicant_income_percent_median'] < 120)] = 'Middle'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df6.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df6.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp6 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp6.ix[:, ["count","%count","amount","%amount",]]]
totalapp6 = pd.concat(pieces, axis=1, keys=['Loan purchased by institution'])
totalapp6

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan purchased by institution  \
                                                                       count   
Loan Purpose and Type Home Improvement                                     3   
                      Purchase - Conventional                             25   
                      Purchase - Goverment                               154   
                      Refinancing                                        144   
Applicant Race/Eth    aian                                                 3   
                      as                                                  15   
                      b                                                    9   
                      h                                                   78   
                      pi                                                   6   
                      w                                                  175   
Applicant Income      Income Not Available                                43   
                      Low                                                  8   
                      Middle                                             104   
                      Moderate                                            74   
                      Upper                                               97   
Applicant Sex         1                                                  243   
                      2                                                   71   
                      3                                                   12   

                                                                        
                                               %count  amount  %amount  
Loan Purpose and Type Home Improvement            0.9     964      1.0  
                      Purchase - Conventional     7.7   12195     13.1  
                      Purchase - Goverment       47.2   42166     45.4  
                      Refinancing                44.2   37632     40.5  
Applicant Race/Eth    aian                        0.9    1500      1.6  
                      as                          4.6    5366      5.8  
                      b                           2.8    2429      2.6  
                      h                          23.9   19344     20.8  
                      pi                          1.8    1662      1.8  
                      w                          53.7   50572     54.4  
Applicant Income      Income Not Available       13.2   15531     16.7  
                      Low                         2.5     918      1.0  
                      Middle                     31.9   26795     28.8  
                      Moderate                   22.7   14098     15.2  
                      Upper                      29.8   35615     38.3  
Applicant Sex         1                          74.5   69567     74.8  
                      2                          21.8   19316     20.8  
                      3                           3.7    4074      4.4

In [61]:
#Merging ALL
total = pd.concat([totalapp,totalapp1,totalapp2,totalapp3,totalapp4,totalapp5,totalapp6], axis=1)
total

App approved but not accepted  \
                                                                       count   
Applicant Income      Income Not Available                                33   
                      Low                                                 48   
                      Middle                                              34   
                      Moderate                                            47   
                      Upper                                               96   
Applicant Race/Eth    aian                                                 1   
                      as                                                  27   
                      b                                                    1   
                      h                                                   23   
                      pi                                                   3   
                      w                                                  154   
Applicant Sex         1                                                  NaN   
                      2                                                  NaN   
                      3                                                  NaN   
                      4                                                  NaN   
Loan Purpose and Type Home Improvement                                     1   
                      Purchase - Conventional                             43   
                      Purchase - Goverment                               NaN   
                      Refinancing                                        214   

                                                                        \
                                               %count  amount  %amount   
Applicant Income      Income Not Available       12.8   12011     11.6   
                      Low                        18.6    9697      9.3   
                      Middle                     13.2   11598     11.2   
                      Moderate                   18.2   11009     10.6   
                      Upper                      37.2   59467     57.3   
Applicant Race/Eth    aian                        0.4     310      0.3   
                      as                         10.5    7890      7.6   
                      b                           0.4     175      0.2   
                      h                           8.9    5658      5.5   
                      pi                          1.2     966      0.9   
                      w                          59.7   64096     61.8   
Applicant Sex         1                           NaN     NaN      NaN   
                      2                           NaN     NaN      NaN   
                      3                           NaN     NaN      NaN   
                      4                           NaN     NaN      NaN   
Loan Purpose and Type Home Improvement            0.4     546      0.5   
                      Purchase - Conventional    16.7   31246     30.1   
                      Purchase - Goverment        NaN     NaN      NaN   
                      Refinancing                82.9   71990     69.4   

                                               Total Applications (1)          \
                                                                count  %count   
Applicant Income      Income Not Available                       1462    14.7   
                      Low                                        1495    15.0   
                      Middle                                     1699    17.1   
                      Moderate                                   1828    18.4   
                      Upper                                      3462    34.8   
Applicant Race/Eth    aian                                         40     0.4   
                      as                                         1023    10.3   
                      b                                           201     2.0   
 

In [45]:
#boa.to_csv('boa.csv')